In [4]:
import sys
import os

# Assuming the notebook is in 'app/lab' and the module is in 'app/app'
module_path = os.path.abspath(os.path.join('..', 'app'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
from ingest import ingest

In [6]:
ingest.teste()

aaaaa


In [10]:
from unstructured.partition.auto import partition
elements = partition("../../docs/Cadastramento inicial no PJe.pdf")

[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vscode/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [13]:
elements

In [16]:
print("\n".join([str(el) for el in elements]))

26/04/2024, 15:43
Cadastramento inicial no PJe
Quando o advogado acessa o PJe pela primeira vez utilizando seu certificado digital o sistema buscará alguns dados diretamente do cadastro da OAB e apresentará uma tela solicitando a complementação do cadastro. Por fim, será apresentado um termo de compromisso, o qual deverá ser assinado digitalmente. Após realizar a assinatura digital do termo de compromisso não é necessário entregá-lo ou comparecer a uma unidade judiciária, bastando que o advogado acesse novamente o PJe com seu certificado digital, quando então terá acesso ao seu painel, podendo protocolar novos processos ou solicitar sua habilitação em processos já existentes.
Procuradores públicos devem ser cadastrados no PJe pelo procurador-gestor da respectiva procuradoria. Para solicitar o cadastramento de nova procuradoria ou suporte, envie e-mail para o endereço seateco@trt4.jus.br, da Secretaria de Apoio Técnico da Corregedoria.
Peritos já cadastrados no CPTEC-TRT4 devem acessar 

In [9]:
path = '../../docs'
for path, dirs, files in os.walk(path):
    print(path)
    for file in files:
        print(file)

../../docs
Cadastramento inicial no PJe.pdf
guiapje-advogados.pdf
guiapje-tribunal.pdf
guiapje-usuarios.pdf
Instalação do PJePortable v. 1.13.10 - para uso do PJe.pdf
MANUAL FUNCIONAMENTO DAS PROCURADORIAS NO PJE.pdf
Manual.pdf
Manual_de_utilização_do_PJe_sem_certificado_digital.pdf
PJE - Processo Judicial Eletrônico.pdf
PJe Mídias Desktop - Envio de Perícias em MP4.pdf
PJeMídias - Acesso à gravação de audiências ou perícias.pdf
PJeMídiasDesktop - Envio de Provas em MP4.pdf
PROCEDIMENTOS PARA ENVIO DE CÁLCULOS DO PJECALC NO PJE PARA ADVOGADOS.pdf
processo_judicial_eletronico_grafica2.pdf
TRT04 - Guia para participação em audiências_sessões com a ferramenta Zoom.pdf
TRT04 - Manual para advogados e peritos (cadastramento de dados bancários para créditos em processos do TRT04).pdf
TRT04 - MANUAL PJE 2.10.1 - PAINEL DO ADVOGADO.pdf
TRT04 - PJE 2.10.1 - Manual de autuação no  painel do procurador.pdf
Zoom - CDTI OAB.pdf
